In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from collections import Counter

In [2]:
train_dataset = np.loadtxt(open('Train Set/kddtrain_2class_normalized.csv'), delimiter=',', dtype=np.float32, skiprows=1)
train_labels = np.loadtxt(open('Train Set/train_labels.csv'), delimiter=',', dtype=np.float32, skiprows=1)
test_dataset = np.loadtxt(open('Test Set/KDDTest+_normalized_2.csv'), delimiter=',', dtype=np.float32, skiprows=1)
test_labels = np.loadtxt(open('Test Set/test_labels.csv'), delimiter=',', dtype=np.float32, skiprows=1)
train_labels = train_labels - 1
test_labels = test_labels - 1
# train_indices = np.loadtxt(open('Minimized Train Set/SelectedRows.csv'), delimiter=',', dtype=np.int32, skiprows=1)
# train_indices -= 2
# train_dataset = train_dataset[train_indices]
# train_labels = train_labels[train_indices]

In [3]:
num_feature = len(train_dataset[0])
num_labels = 2
# scaler = MinMaxScaler()
# scaler.fit(train_dataset)
# train_dataset = scaler.fit_transform(train_dataset)
# test_dataset = scaler.fit_transform(test_dataset)
# l = float(len(train_dataset)) + float(len(test_dataset))
# for i in range(num_feature):
#     if len(set(train_dataset[:, i])) > 2:
#         s = float(sum(train_dataset[:, i]) + sum(test_dataset[:, i]))
#         avg = s / l
#         train_dataset[:, i] = (train_dataset[:, i] - (avg / 2)) / 2
#         test_dataset[:, i] = (test_dataset[:, i] - (avg / 2)) / 2

In [4]:
len(train_dataset)

125973

In [5]:
valid_dataset = train_dataset[-6670:, :]
valid_labels = train_labels[-6670:]
train_dataset = train_dataset[:-6670, :]
train_labels = train_labels[:-6670]

In [6]:
train_labels.shape

(119303,)

In [7]:


def reformat(dataset, labels):
  dataset = dataset.reshape((-1, num_feature)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
# test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Testing set', test_dataset.shape, test_labels.shape


Training set (119303, 41) (119303, 2)
Validation set (6670, 41) (6670, 2)
Testing set (22544, 41) (22544, 2)


In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [9]:
batch_size = 2000

graph = tf.Graph()
with graph.as_default():


    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, num_feature))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    n_hidden_1 = 200
    n_hidden_2 = 100
    n_hidden_3 = 10
    n_input = num_feature

    wh1 = tf.Variable(tf.truncated_normal([n_input, n_hidden_1], dtype=tf.float32))
    wh2 = tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], dtype=tf.float32))
    wh3 = tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], dtype=tf.float32))
    wo = tf.Variable(tf.truncated_normal([n_hidden_3, num_labels], dtype=tf.float32))
    b1 = tf.Variable(tf.truncated_normal([n_hidden_1], dtype=tf.float32))
    b2 = tf.Variable(tf.truncated_normal([n_hidden_2], dtype=tf.float32))
    b3 = tf.Variable(tf.truncated_normal([n_hidden_3], dtype=tf.float32))
    bo = tf.Variable(tf.truncated_normal([num_labels], dtype=tf.float32))

    def compute_logit(x):
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, wh1), b1))
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, wh2), b2))
        layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, wh3), b3))
        return tf.matmul(layer_3, wo) + bo

    
    logits = compute_logit(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.0001 * (tf.nn.l2_loss(wh1) + tf.nn.l2_loss(wh2) + tf.nn.l2_loss(wo))

    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(compute_logit(tf_valid_dataset))
    test_prediction = tf.nn.softmax(compute_logit(tf_test_dataset))

In [ ]:
num_steps = 10000
preds = []
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print "Minibatch loss at step", step, ":", l
            bac = accuracy(predictions, batch_labels)
            print "Minibatch accuracy: %.1f%%" % bac
            vac = accuracy(valid_prediction.eval(), valid_labels) 
            print "Validation accuracy: %.1f%%" % vac

    print "Minibatch loss at step", step, ":", l
    print "Minibatch accuracy: %.3f%%" % accuracy(predictions, batch_labels)
    print "Validation accuracy: %.3f%%" % accuracy(valid_prediction.eval(), valid_labels)
    preds = test_prediction.eval()
    print "Test accuracy: %.3f%%" % accuracy(preds, test_labels)
    

Initialized
Minibatch loss at step 0 : 1.95225
Minibatch accuracy: 41.6%
Validation accuracy: 45.7%
Minibatch loss at step 500 : 0.747367
Minibatch accuracy: 97.8%
Validation accuracy: 97.7%
Minibatch loss at step 1000 : 0.473955
Minibatch accuracy: 98.2%
Validation accuracy: 98.0%
Minibatch loss at step 1500 : 0.321225
Minibatch accuracy: 98.5%
Validation accuracy: 98.1%
Minibatch loss at step 2000 : 0.256912
Minibatch accuracy: 98.2%
Validation accuracy: 98.5%
Minibatch loss at step 2500 : 0.207983
Minibatch accuracy: 98.8%
Validation accuracy: 98.5%
Minibatch loss at step 3000 : 0.193433
Minibatch accuracy: 98.5%
Validation accuracy: 98.8%
Minibatch loss at step 3500 : 0.171717
Minibatch accuracy: 99.0%
Validation accuracy: 98.7%
Minibatch loss at step 4000 : 0.153105
Minibatch accuracy: 99.0%
Validation accuracy: 98.9%
Minibatch loss at step 4500 : 0.140647
Minibatch accuracy: 99.3%
Validation accuracy: 98.9%
Minibatch loss at step 5000 : 0.144159
Minibatch accuracy: 98.8%
Validati

In [16]:
preds_ = [int(round(x[0])) for x in preds]

In [22]:
confusion_matrix(test_labels[:,0], preds_)

array([[9477,  234],
       [4945, 7888]])

In [23]:
test_dataset.describe()

AttributeError: 'numpy.ndarray' object has no attribute 'describe'

In [26]:
Counter(test_labels[:,0])

Counter({1.0: 12833, 0.0: 9711})

In [27]:
test_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       ..., 
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.]], dtype=float32)